## Computation Graph

### Operations

In [1]:
class Operation():
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
        
    def compute(self):
        pass

In [2]:
class add(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [3]:
class multiply(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [4]:
class matmul(Operation):
    def __init__(self, a, b):
        super().__init__([a,b])
        
    def compute(self, a_mat, b_mat):
        self.inputs = [a_mat, b_mat]
        return a_mat.dot(b_mat)

### Elements

In [5]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [6]:
class Variable():
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)

In [7]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

### Example Graph

z = Ax + B

with A = 10 and B = 1

In [8]:
g = Graph()

In [9]:
g.set_as_default()

In [10]:
A = Variable(10)

In [11]:
B = Variable(1)

In [12]:
X = Placeholder()

In [13]:
y = multiply(A, X)

In [14]:
z = add(y , B)

### Session

In [29]:
import numpy as np

In [15]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [22]:
class Session:
    
    def run(self, operation, feed_dict = {}):

        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output

In [23]:
sess = Session()

In [24]:
result = sess.run(operation=z, feed_dict={X:10})

In [25]:
result

101

In [26]:
g = Graph()

g.set_as_default()

A = Variable([[10,20],[30,40]])
b = Variable([1,1])

x = Placeholder()

y = matmul(A,x)

z = add(y,b)

In [27]:
sess = Session()

In [30]:
result = sess.run(operation=z,feed_dict={x:10})

In [31]:
result

array([[101, 201],
       [301, 401]])